# Task 1 - Analyze the dataset
Parse the groundtruth information into a dictionary. The key will be the name of the picture and the value a list of the signals and their information. 

This data structure let us find directly the picture that we want to process.

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Enable matplotline inline display
%matplotlib inline

# Import built-in modules
import os

# Import third party modules
import imageio
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import local functions
from utils import get_files_from_dir, get_gt_data, get_img, get_n_samples_of_col, \
    get_unique_values_of_col, get_patch, gt_to_img, gt_to_mask, parse_gt_data, ROOT_DIR


In [ ]:

# Useful directories
TRAIN_DIR = os.path.join(ROOT_DIR, 'dataset', 'train')
TRAIN_GTS_DIR = os.path.join(TRAIN_DIR, 'gt')
TRAIN_MASKS_DIR = os.path.join(TRAIN_DIR, 'mask')
TEST_DIR = os.path.join(ROOT_DIR, 'dataset', 'test')


In [ ]:


gt_filenames = get_files_from_dir(TRAIN_GTS_DIR)
mask_filenames = get_files_from_dir(TRAIN_MASKS_DIR)

data = dict()

for gt in gt_filenames:
    mask_path = gt_to_mask(gt)
    mask_img = get_img(TRAIN_MASKS_DIR, mask_path)
    
    lines = get_gt_data(TRAIN_GTS_DIR, gt)
        
    im_name = gt_to_mask(gt)
    im_open = get_img(TRAIN_MASKS_DIR, im_name)

    lista = list()
    for l in lines:
        tly, tlx, bry, brx, tipo = parse_gt_data(l)

        d = dict()
        d['type'] = tipo.strip()
        
        w = brx - tlx
        h = bry - tly
        
        d['width'] = w
        d['height'] = h
        d['bbox_area'] = w*h
        d['form_factor'] = w/h
        
        d['tly'] = round(tly)
        d['tlx'] = round(tlx)
        d['bry'] = round(bry)
        d['brx'] = round(brx)
        
        mask_patch = get_patch(mask_img, d['tlx'], d['tly'], d['brx'], d['bry'])
        mask_area = np.count_nonzero(mask_patch)
        d['mask_area'] = mask_area
        d['filling_ratio'] = mask_area / d['bbox_area']
        d['mask'] = mask_patch != 0
        
        lista.append(d)
        
    data[gt] = lista


Show information in pandas format to filter the signals by type easily.


In [ ]:
columns = [
    'type', 'width', 'height', 'form_factor', 'tlx', 'tly', 'brx', 'bry', 
    'bbox_area', 'mask_area', 'filling_ratio', 'mask'
]
df = pd.DataFrame.from_dict({(i,n): data[i][n]
                        for i in data.keys()
                        for n,v in enumerate(data[i])}, columns=columns, orient='index').sort_values(['type'])
df

In [ ]:
type_counts = df.groupby('type').aggregate(np.std).round(2)
#type_counts = df.groupby('type').aggregate(np.median)
#type_counts = df.groupby('type').aggregate(np.average)
#type_counts = df.groupby('type').aggregate(np.std)


cmap = mpl.colors.ListedColormap(['red','red','red', 'blue', 'purple', 'blue'])
type_counts['mask_area'].plot(figsize=(12, 8),kind='pie',sort_columns=True, colormap=cmap, title="Number of pixels per class")


### Conclusions: 
Large range of sizes: 
- From 900 pixels (about 30x30) to 56000 of area


Total number of pixels of each class is very different:
- class F: 42%
- class B: 2%

In [ ]:
# Study average, std and median values
type_counts = df.groupby('type').aggregate(np.average).round(2)
print("Average values")
print(type_counts)
type_counts.drop(columns=['width', 'height', 'form_factor'])\
    .plot(figsize=(5, 8), kind='bar', sort_columns=True,subplots=True)
type_counts['filling_ratio'].plot(figsize=(7, 3),kind='bar',sort_columns=True, title="Average filling ratio")

type_counts = df.groupby('type').aggregate(np.std).round(3)
print("\n\nStandard deviation values")
print(type_counts)

type_counts = df.groupby('type').aggregate(np.median).round(2)
print("\n\nMedian values")
print(type_counts)

### Conclusions:
Average and median bounding-box sizes around 90 pixels

Filling_ratio pretty consistent through classes
- Triangles: 0.5
- Circles: 0.77
- Rectangles: 0.99

In [ ]:
# Plot and compare results

# Task 2 - Split training dataset

Extract the 30% of the training images of each class to set up a validation dataset

In [ ]:
# Choose randomly the 70% of each class
# Count number of signals per class
signal_types = get_unique_values_of_col(df, 'type')
n_signals = df['type'].value_counts(sort=False).reindex(signal_types)
print(n_signals)
n_signals = df['type'].value_counts(sort=False).reindex(signal_types).aggregate(sum)
print("\n\nNumber of signals: {}".format(n_signals))

# Plot
df['type']\
    .value_counts(sort=False)\
    .reindex(signal_types)\
    .plot(figsize=(10, 7),kind='bar',sort_columns=True, title="Number of images per class", grid=True)


In [ ]:
# Small captions of each type
plt.figure(figsize=(20, 20))
n_img = 5
types = get_unique_values_of_col(df, 'type')
l_types = len(types)

for t_idx, t in enumerate(types):
    samples_gt = get_n_samples_of_col(df, 'type', n_img, t)
    for gt_idx, gt in enumerate(samples_gt):
        train_name = gt_to_img(gt)

        row = df.loc[gt]
        row_data = row[row['type'] == t].iloc[0]

        img = get_img(TRAIN_DIR, train_name)
        img_patch = get_patch(img, row_data['tlx'], row_data['tly'], row_data['brx'], row_data['bry'])

        plt.subplot(l_types, n_img, gt_idx + 1 + t_idx*n_img)
        plt.xticks([])
        plt.yticks([])
        plt.title('{type}: {filename}'.format(type=t, filename=train_name))
        plt.imshow(img_patch * row_data['mask'][..., np.newaxis])

In [ ]:
# Get only one class
df_filtered = df[df['type'] == "B"].drop(columns='mask')

In [ ]:
df_sorted_train = df_filtered.sample(frac=0.3)
print(df_sorted_train)


In [ ]:
tuple(zip(df_sorted_train.index.get_level_values(0).tolist(),df_sorted_train.index.get_level_values(1).tolist()))


In [ ]:
df_sorted_test = pd.concat([df_filtered, df_sorted_train]).drop_duplicates(keep=False)
print(df_sorted_test)

We will obtain different list:
- train_images_red: Train selection of red signals
- train_images_blue: Train selection of blue signals
- train_images_mix: Train selection of red/blue signals
- train_images: Total train selection
- val_images: Total validation selection

In [ ]:
#Functions to do all the process in one cell
def split_class(signal_class, train_percentage):
    
    # Choose one signal class
    df_filtered = df[df['type'] == signal_class].drop(columns=['mask'])
    
    # Sample randomly the percentage choosen
    df_sorted_train = df_filtered.sample(frac=train_percentage)
    
    # Save train selection
    train_images = tuple(zip(df_sorted_train.index.get_level_values(0).tolist(),df_sorted_train.index.get_level_values(1).tolist()))
    
    # Delete train selection to get validation selection
    df_sorted_test = pd.concat([df_filtered,df_sorted_train]).drop_duplicates(keep=False)
    val_images = tuple(zip(df_sorted_test.index.get_level_values(0).tolist(),df_sorted_test.index.get_level_values(1).tolist()))
    
    return train_images, val_images

def split_dataset(data,percentage, classes):
    
    train_images = []
    val_images = []
    
    for signal_class in classes:
        temp_train_images, temp_val_images = split_class(signal_class, percentage)
        train_images += temp_train_images
        val_images += temp_val_images
    return train_images, val_images


classes_red = ['A','B','C']
classes_blue = ['D','F']
classes_mix = ['E']

train_images_red, val_images_red = split_dataset(data, 0.7, classes_red)
train_images_blue, val_images_blue = split_dataset(data, 0.7, classes_blue)
train_images_mix, val_images_mix = split_dataset(data, 0.7, classes_mix)

train_images = train_images_red + train_images_blue + train_images_mix
val_images = val_images_red + val_images_blue + val_images_mix
    

# Task 3 - Separation by colour


For each color group of images (_Red, Blue, Mix_) we calculate the histograms in different color spaces.
The goal is to build a color mask applying the propper thresholds. We will obtain this information from the histograms.


# Calculate histograms:

In [ ]:
from skimage import color
from skimage import exposure
np.seterr(divide='ignore', invalid='ignore')

dirs_jpg = get_files_from_dir(TRAIN_DIR)

##############################
# Calculate color histograms:
##############################

def color_histogram(gt_dictionary, train_selec, path_jpg, color_space):
    
    '''gt_dictionary = Dictionary containing ground truth data
       train selec = Group of training images divided by color (Red, Blue, Mix)
       path_jpg = Path to training images 
       color_space = Color space in which to perform histogram (rgb, hsv, ycbcr, xyz)'''

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)

    r_hist = np.zeros(255)
    g_hist = np.zeros(255)
    b_hist = np.zeros(255)

    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(TRAIN_MASKS_DIR, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        if color_space=='rgb':
            final_roi = jpg_roi
            r0 = 0
            rf = 255

        elif color_space=='hsv':
            final_roi = color.rgb2hsv(jpg_roi) 
            r0 = 0
            rf = 1

        elif color_space=='ycbcr':
            final_roi = color.rgb2ycbcr(jpg_roi)
            r0 = 0
            rf = 255

        elif color_space=='xyz':
            final_roi = color.rgb2xyz(jpg_roi)
            r0 = 0
            rf = 1

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(r0,rf))[1]
        c0_hist += np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(r0,rf))[0]
        c1_hist += np.histogram(final_roi[:,:,1] * mask_roi, bins=255, range=(r0,rf))[0]
        c2_hist += np.histogram(final_roi[:,:,2] * mask_roi, bins=255, range=(r0,rf))[0]

    return bins, c0_hist, c1_hist, c2_hist, r0, rf

def hsv_histogram(gt_dictionary, path_jpg):

    h_hist = np.zeros(255)
    s_hist = np.zeros(255)
    v_hist = np.zeros(255)

    for gt, values in list(gt_dictionary.items()):
        for v in values:

            jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
            jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
            hsv_roi = color.rgb2hsv(jpg_roi)*255
            
            bins = np.histogram(hsv_roi[:,:,0], bins=255, range=(1,255))[1]
            h_hist += np.histogram(hsv_roi[:,:,0], bins=255, range=(1,255))[0]
            s_hist += np.histogram(hsv_roi[:,:,1], bins=255, range=(1,255))[0]
            v_hist += np.histogram(hsv_roi[:,:,2], bins=255, range=(1,255))[0]
    
    return bins, h_hist, s_hist, v_hist

######################################
# Calculate RGB normalized histogram:
######################################

def norm_histogram(gt_dictionary, train_selec, path_jpg):

    '''gt_dictionary = Dictionary containing ground truth data
       train selec = Group of training images divided by color (Red, Blue, Mix)
       path_jpg = Path to training images'''

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)
    
    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        # Important: in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
        # we will be training our algorithm poorly.After trying this, we saw all images have saturated
        # pixels (aka normalized image = original image). 

        # We will normalize the ROIs, knowing this is poorly training the algorithm.

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(TRAIN_MASKS_DIR, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        jpg_max_0 = np.max(jpg_roi[:,:,0])
        jpg_max_1 = np.max(jpg_roi[:,:,1])
        jpg_max_2 = np.max(jpg_roi[:,:,2])

        final_roi = jpg_roi
        jpg_max_0, jpg_max_1, jpg_max_2 = jpg_max_0, jpg_max_1, jpg_max_2 

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(0,1))[1]
        c0_hist += np.histogram(final_roi[:,:,0] / jpg_max_0 * mask_roi, bins=255, range=(0,1))[0]
        c1_hist += np.histogram(final_roi[:,:,1] / jpg_max_1 * mask_roi, bins=255, range=(0,1))[0]
        c2_hist += np.histogram(final_roi[:,:,2] / jpg_max_2 * mask_roi, bins=255, range=(0,1))[0]

    return bins, c0_hist, c1_hist, c2_hist


###################
# Plot histograms:
###################

def plot_histogram(hist0, hist1, hist2, r0, rf, color_name):
    
    '''hist0, hist1, hist2 = Histograms corresponding to channels 0, 1, 2 of the images
       r0, r1 = Min and Max values of the selected color space
       color_name = Name of the resulting plot'''
    
    path_figures = os.path.join('figures')
    try:
        os.stat(path_figures)
    except:
        os.mkdir(path_figures)

    x = np.linspace(r0, rf, 255)
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
    
    axs[0].bar(x[:-2], hist0[1:-1], color='r', width=0.8*(rf-r0)/255, label='Ch1')
    axs[1].bar(x[:-2], hist1[1:-1], color='g', width=0.8*(rf-r0)/255, label='Ch2')
    axs[2].bar(x[:-2], hist2[1:-1], color='b', width=0.8*(rf-r0)/255, label='Ch3')

    axs[0].legend()
    axs[1].legend()
    axs[2].legend()

    axs[0].set_xlabel('8bit quantification')
    axs[1].set_xlabel('8bit quantification')
    axs[2].set_xlabel('8bit quantification')
    axs[0].set_ylabel('Total number of px')
    
    fig.suptitle(color_name + ' histogram')
    plt.savefig('figures/' + str(color_name) + '_hist.png')
    plt.show()


# Calculate histograms in different color spaces:

We split the signals by color and calculate the histogram of each group:


RGB histograms:

In [ ]:
bins, r_hist_red, g_hist_red, b_hist_red, rgb0, rgbf = color_histogram(data, train_images_red, TRAIN_DIR, 'rgb')
bins, r_hist_blue, g_hist_blue, b_hist_blue, rgb0_blue, rgbf = color_histogram(data, train_images_blue, TRAIN_DIR, 'rgb')
bins, r_hist_mix, g_hist_mix, b_hist_mix, rgb0, rgbf = color_histogram(data, train_images_mix, TRAIN_DIR, 'rgb')

Normalized RGB histograms:

**Important:** in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
we will be training our algorithm poorly. After trying this, we saw all images have saturated
pixels (aka normalized image = original image). 

We will normalize the ROIs, knowing this is poorly training the algorithm.

In [ ]:
bins_norm, r_hist_norm_red, g_hist_norm_red, b_hist_norm_red = norm_histogram(data, train_images_red, TRAIN_DIR)
bins_norm, r_hist_norm_blue, g_hist_norm_blue, b_hist_norm_blue = norm_histogram(data, train_images_blue, TRAIN_DIR)
bins_norm, r_hist_norm_mix, g_hist_norm_mix, b_hist_norm_mix = norm_histogram(data, train_images_mix, TRAIN_DIR)


HSV histograms:


In [ ]:
hbins, h_hist_red, s_hist_red, v_hist_red, hsv0, hsvf = color_histogram(data, train_images_red, TRAIN_DIR, 'hsv')
hbins, h_hist_blue, s_hist_blue, v_hist_blue, hsv0, hsvf = color_histogram(data, train_images_blue, TRAIN_DIR, 'hsv')
hbins, h_hist_mix, s_hist_mix, v_hist_mix, hsv0, hsvf = color_histogram(data, train_images_mix, TRAIN_DIR, 'hsv')

yCbCr histograms:

In [ ]:
ybins, y_hist_red, cb_hist_red, cr_hist_red, ycbcr0, ycbcrf = color_histogram(data, train_images_red, TRAIN_DIR, 'ycbcr')
ybins, y_hist_blue, cb_hist_blue, cr_hist_blue, ycbcr0, ycbcrf = color_histogram(data, train_images_blue, TRAIN_DIR, 'ycbcr')
ybins, y_hist_mix, cb_hist_mix, cr_hist_mix, ycbcr0, ycbcrf = color_histogram(data, train_images_mix, TRAIN_DIR, 'ycbcr')

XYZ histograms:

In [ ]:
xbins, xx_hist_red, yy_hist_red, zz_hist_red, xyz0, xyzf = color_histogram(data, train_images_red, TRAIN_DIR, 'xyz')
xbins, xx_hist_blue, yy_hist_blue, zz_hist_blue, xyz0, xyzf = color_histogram(data, train_images_blue, TRAIN_DIR, 'xyz')
xbins, xx_hist_mix, yy_hist_mix, zz_hist_mix, xyz0, xyzf = color_histogram(data, train_images_mix, TRAIN_DIR, 'xyz')

# Plot histograms in different color spaces:

Plot RGB histograms:

In [ ]:
plot_histogram(r_hist_red, g_hist_red, b_hist_red, rgb0, rgbf, 'RGB - Red Signals')
plot_histogram(r_hist_blue, g_hist_blue, b_hist_blue, rgb0, rgbf, 'RGB - Blue Signals')
plot_histogram(r_hist_mix, g_hist_mix, b_hist_mix, rgb0, rgbf, 'RGB - Mixed Signals')

Plot normalized RGB histograms:

In [ ]:
plot_histogram(r_hist_norm_red, g_hist_norm_red, b_hist_norm_red, rgb0, rgbf, 'RGB Norm - Red Signals')
plot_histogram(r_hist_norm_blue, g_hist_norm_blue, b_hist_norm_blue, rgb0, rgbf, 'RGB Norm - Blue Signals')
plot_histogram(r_hist_norm_mix, g_hist_norm_mix, b_hist_norm_mix, rgb0, rgbf, 'RGB Norm - Mixed Signals')

Plot HSV histograms:

In [ ]:
plot_histogram(h_hist_red, s_hist_red, v_hist_red, hsv0, hsvf, 'HSV - Red signals')
plot_histogram(h_hist_blue, s_hist_blue, v_hist_blue, hsv0, hsvf, 'HSV - Blue signals')
plot_histogram(h_hist_mix, s_hist_mix, v_hist_mix, hsv0, hsvf, 'HSV - Mixed signals')

Plot yCbCr histograms:

In [ ]:
plot_histogram(y_hist_red, cb_hist_red, cr_hist_red, ycbcr0, ycbcrf, 'yCbCr - Red signals')
plot_histogram(y_hist_blue, cb_hist_blue, cr_hist_blue, ycbcr0, ycbcrf, 'yCbCr - Blue signals')
plot_histogram(y_hist_mix, cb_hist_mix, cr_hist_mix, ycbcr0, ycbcrf, 'yCbCr - Mixed signals')

Plot XYZ histograms:

In [ ]:
plot_histogram(xx_hist_red, yy_hist_red, zz_hist_red, xyz0, xyzf, 'XYZ - Red signals')
plot_histogram(xx_hist_blue, yy_hist_blue, zz_hist_blue, xyz0, xyzf, 'XYZ - Blue signals')
plot_histogram(xx_hist_mix, yy_hist_mix, zz_hist_mix, xyz0, xyzf, 'XYZ - Mixed signals')

# Normalized RGB histogram:

In [ ]:
from skimage import color

dirs_jpg = get_files_from_dir(TRAIN_DIR)

def norm_histogram(gt_dictionary, train_selec, path_jpg, color_space):

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)
    
    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        # Important: in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
        # we will be training our algorithm poorly.After trying this, we saw all images have saturated
        # pixels (aka normalized image = original image). 

        # We will normalize the ROIs, knowing this is poorly training the algorithm.

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(TRAIN_MASKS_DIR, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        jpg_max_0 = np.max(jpg_roi[:,:,0])
        jpg_max_1 = np.max(jpg_roi[:,:,1])
        jpg_max_2 = np.max(jpg_roi[:,:,2])

        if color_space=='rgb':
            final_roi = jpg_roi
            jpg_max_0, jpg_max_1, jpg_max_2 = jpg_max_0, jpg_max_1, jpg_max_2 

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(0,1))[1]
        c0_hist += np.histogram(final_roi[:,:,0] / jpg_max_0 * mask_roi, bins=255, range=(0,1))[0]
        c1_hist += np.histogram(final_roi[:,:,1] / jpg_max_1 * mask_roi, bins=255, range=(0,1))[0]
        c2_hist += np.histogram(final_roi[:,:,2] / jpg_max_2 * mask_roi, bins=255, range=(0,1))[0]

    return bins, c0_hist, c1_hist, c2_hist


################################################
# Calculate histograms with traffic signal data:
################################################

# RGB histograms:

bins_norm, r_hist_norm, g_hist_norm, b_hist_norm = norm_histogram(data, train_images, TRAIN_DIR, 'rgb')



# Plot normalized histogram:

In [ ]:
#plot_histogram(r_hist_norm, 0, 1, 'Red_norm', 'r')
#plot_histogram(g_hist_norm, 0, 1, 'Green_norm', 'g')
#plot_histogram(b_hist_norm, 0, 1, 'Blue_norm', 'b')

Comparar las máscaras obtenidas con el ground truth.

Podemos ver si sirven las funciones que ya nos dieron hechas.



## Conclusions: 

1. HSV & yCbCr are better than RGB or XYZ for color segmentation.

2. Selected thresholds: 
    - RGB: Red: r > 0.03; 0.46 < r < 0.51; 0.98 < r
    - RGB: Blue: 0.59 < b < 0.62
    - yCbCr: Cb = 0.4 < cb < 0.65
    - yCbCr: Cr = 0.38 < Cr < 0.7
    
3. We only study the colors of the signals, so we expect to have False Positives of the same colors in the background